In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
from sklearn.utils import shuffle
print('import done')

C:\Users\Javapocalypse\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


import done


In [45]:
dataset_train = "Dataset1/queensland/train.xlsx"
dataset_test = "Dataset1/queensland/test.xlsx"

In [46]:
train = pd.read_excel(dataset_train)
test = pd.read_excel(dataset_test)

train = train.sample(frac=1)

print(train.head())
print('-'*30)
print(train['label'].value_counts())
print('-'*30)

# train = train[train['text'].str.len()  15]

# print('x'*50)
# # df[df['column name'].map(len) < 2]
# print(train.head())
# print('-'*30)
# print(train['label'].value_counts())


                tweet_id                                               text  \
676   295401764763611008  ARAI MOTOR BIKE HELMETBLACK ALMOST BRAND NEW 2...   
1699  296087615508185024  EVERYONE STOP WHAT YOURE DOING LIKE s new FB P...   
2144  297248827335905024      The lake university at Southbank Brisbane pic   
5957  295757652447875008  Tony Abbott visiting flood affected areas Quee...   
2880  295782312292006976                    Flood worsens eastern Australia   

             label  
676   not_relevant  
1699  not_relevant  
2144  not_relevant  
5957      relevant  
2880      relevant  
------------------------------
relevant        3248
not_relevant    2771
Name: label, dtype: int64
------------------------------


In [47]:
# train['text'][0]
# train['label'][0]
dataColumn = 'text'
labelColumn = 'label'

In [48]:
tags = train[labelColumn]
texts = train[dataColumn]

In [49]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags.astype(str))
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)

[0 0 0 1 1]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
(6019,) (6019, 1000)


In [50]:
# for cnn preproces
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)

[947, 9, 14]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0 947   9  14]
(6019, 100)


In [51]:
def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

In [52]:
def get_simple_model():
#     model = Sequential()
#     model.add(Dense(512, activation='relu', input_shape=(num_max,)))
#     model.add(Dropout(0.2))
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))
#     model.summary()
#     model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['acc',metrics.binary_accuracy])

    model = Sequential()
    model.add(Dense(512, input_shape=(num_max,)))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
    print('compile done')
    return model

In [53]:
m = get_simple_model()
check_model(m,mat_texts,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 512)               512512    
_________________________________________________________________
activation_10 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 513       
_________________________________________________________________
activation_11 (Activation)   (None, 1)                 0         
Total params: 513,025
Trainable params: 513,025
Non-trainable params: 0
_________________________________________________________________
compile done
Train on 4815 samples, validate on 1204 samples
Epoch 1/10
4815/4815 [==============================] - 1s 221us/step - loss: 7.3702 - acc: 0.5377 - val_loss: 7.2164 - val_acc: 0.5473
Epoch 2/10
4815/4815 [==============================] - 1s 168us/step -

In [54]:
def get_cnn_model_v1():   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [55]:
m = get_cnn_model_v1()
check_model(m,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 98, 64)            3904      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_12 (Activation)   (None, 256)               0         
__________

In [56]:
def get_cnn_model_v2(): # added embed   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        50, #!!!!!!!!!!!!!!!!!!!!!!!
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [57]:
m = get_cnn_model_v2()
check_model(m,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 64)            9664      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_14 (Activation)   (None, 256)               0         
__________